In [1]:
import shadow_reconstruction
import qutip as qt
import numpy
from matplotlib import pyplot
import os

In [2]:
def make_perms(n):
    return itertools.permutations(numpy.arange(n))

def gen_cliffords():
    yield from qt.qubit_clifford_group()


def get_num_cons_cliffords():
    def f(x):
        y = numpy.eye(4, dtype=numpy.complex128)
        y[1:3, 1:3] = x
        return y
    return {
        k: f(v.__array__()) for k, v in enumerate(gen_cliffords())
    }

In [3]:
cliffords = numpy.array(list(get_num_cons_cliffords().values()))

/tmp/ipykernel_106278/2038288818.py:5: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  yield from qt.qubit_clifford_group()


# Simple to start

$$ \vert \psi \rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$$

In [4]:
zz = numpy.array([[0.,1.,1.,0.]])
zz /= numpy.linalg.norm(zz)

rho = zz.T @ zz
# pyplot.imshow(rho)
# pyplot.show()

print(numpy.trace(rho))
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))

0.9999999999999998


In [5]:
# rho = shadow_reconstruction.DensityMatrix.new_pure_dense(zz[0,:].astype(numpy.complex128))

In [6]:
# exp = shadow_reconstruction.Experiment(2, ops=cliffords)

In [7]:
# exp.sample(rho, 10)

In [8]:
# if os.path.exists('samples_pure.dat'):
#     print("Loading...")
#     samples = shadow_reconstruction.Samples.load_from('samples.dat')
# else:
#     print("Sampling...")
#     samples = exp.sample(rho, 1000)
#     samples.save_to('samples.dat')

In [9]:
# recon = shadow_reconstruction.Reconstruction()

In [10]:
# recon.estimate_operator_string("ZZ", samples)

In [11]:
# recon.estimate_operator_string("+-", samples)

In [12]:
# recon.estimate_operator_string("-+", samples)

# Bigger

$$|\psi\rangle = \frac{|1000> + |0001>}{\sqrt{2}}$$

In [13]:
# (01 + 10)00
zz = numpy.zeros((1, 1<<4), dtype=numpy.complex128)
zz[0, 0b1000] = 1.0
zz[0, 0b0100] = 1.0
zz /= numpy.linalg.norm(zz)
rho = zz.T @ zz
# print(numpy.trace(rho))
rho /= numpy.trace(rho)
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-1+0j))

In [14]:
rho = shadow_reconstruction.DensityMatrix.new_pure_dense(zz[0,:].astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-0.9999999999999998+0j))

In [15]:
exp = shadow_reconstruction.Experiment(4, ops=cliffords)

In [16]:
# if os.path.exists('larger_samples_pure.dat'):
#     print("Loading...")
#     samples = shadow_reconstruction.Samples.load_from('larger_samples_pure.dat')
# else:
print("Sampling...")
samples = exp.sample(rho, 100000)
print("Saving...")
samples.save_to('larger_samples_pure.dat')

Sampling...
Saving...


In [17]:
recon = shadow_reconstruction.Reconstruction()

In [18]:
recon.use_smart_estimator()
recon.estimate_string("ZIII", samples), rho.expectation_string('ZIII')

((0.006620000000000026+0j), 0j)

In [19]:
recon.use_smart_estimator()
recon.estimate_string("ZZII", samples), rho.expectation_string('ZZII')

((-0.9999179999999999+0j), (-0.9999999999999998+0j))

In [20]:
recon.use_smart_estimator()
recon.estimate_string("ZIZI", samples), rho.expectation_string('ZIZI')

((0.004787999999999999+0j), 0j)

In [21]:
recon.use_smart_estimator()
recon.estimate_string("+-II", samples), rho.expectation_string('+-II')

((0.5020649999999999-0.002384999999999999j), (0.4999999999999999+0j))

In [25]:
recon.use_smart_estimator()
recon.estimate_string("-++-", samples), rho.expectation_string('-++-')

(0j, 0j)